In [1]:
import sys;
sys.path.insert(0, '..')

In [2]:
from math import sqrt, pi, atan2, cos
from util import cis, is_close

## Exercise 1

How can we make the sequence given above into a valid quantum state?

Sequence:

In [3]:
N = 8
sequence = [cis(l*2*pi/N) for l in range(N)]
print([(round(amp.real, 5) + 1j*round(amp.imag, 4)) for amp in sequence])

[(1+0j), (0.70711+0.7071j), 1j, (-0.70711+0.7071j), (-1+0j), (-0.70711-0.7071j), (-0-1j), (0.70711-0.7071j)]


**Answer:**

Multiply each item by $\frac{1}{\sqrt{N}}$

In [4]:
N = 8
sequence_state = [1/sqrt(N) * cis(l*2*pi/N) for l in range(N)]

In [5]:
# check that the squared magnitudes add up to one
assert is_close(sum([abs(i)**2 for i in sequence_state]), 1.0)

## Exercise 2

Show that the encoded angle is reflected in the phase of the amplitude corresponding to outcome 1.

In [6]:
def geom(n, theta):
    N = 2**n
    return [sqrt(1/N) * cis(k*theta) for k in range(N)]

In [7]:
state = geom(1, pi/3)

**Answer:**

In [8]:
assert(round(atan2(state[1].imag, state[1].real), 5) == round(pi/3, 5))

## Exercise 3


In the gate-based implementation of the IQFT, the rotations are applied incrementally with controlled phase rotations. To make it easier to understand the effect of the nested `for` loops in the quantum implementation, below is a classical equivalent of its effect. This function is analogous to the FFT.

In [9]:
from math import log2

def bin_digit(k, j):
    return 1 if k & (1 << j) else 0

def cfft(state):
    n = int(log2(len(state)))
    for j in range(n)[::-1]:
        for k in range(len(state)):
            if bin_digit(k, j) == 0:
                # Compute the sum and difference of the amplitude pair
                state[k] = 1/sqrt(2)*(state[k] + state[k+2**j])
                state[k+2**j] = state[k] - sqrt(2)*state[k+2**j]

            else:
                state[k] *= cis(-pi * (k%2**j)*2**-j)

Verify that the function above gives the same output as the FFT (with a constant $\frac{1}{\sqrt{N}}$ and bit reversal).

**Answer:**

In [10]:
N = 8
frequency = 1.7
samples = [1/sqrt(N)*cos(2 * pi * frequency * (i / N)) for i in range(N)]

In [11]:
import numpy as np
f = np.fft.fft(samples)
f

array([ 0.01813862+0.j        , -0.11373594+0.34545166j,
        0.93169317-0.9912542j ,  0.40521579-0.17396922j,
        0.36394247+0.j        ,  0.40521579+0.17396922j,
        0.93169317+0.9912542j , -0.11373594-0.34545166j])

In [12]:
for i in f:
    adjusted = i*1/sqrt(N)
    print(round(adjusted.real, 5) + 1j*round(adjusted.real, 5))

(0.00641+0.00641j)
(-0.04021-0.04021j)
(0.3294+0.3294j)
(0.14327+0.14327j)
(0.12867+0.12867j)
(0.14327+0.14327j)
(0.3294+0.3294j)
(-0.04021-0.04021j)


In [13]:
cfft(samples)

In [14]:
# reverse the order
n = int(np.log2(len(samples)))
s = samples.copy()
for k in range(len(samples)):
    s[k] = samples[int(bin(k)[2:].zfill(n)[::-1], 2)]

In [15]:
for i in s:
    print(round(i.real, 5) + 1j*round(i.real, 5))

(0.00641+0.00641j)
(-0.04021-0.04021j)
(0.3294+0.3294j)
(0.14327+0.14327j)
(0.12867+0.12867j)
(0.14327+0.14327j)
(0.3294+0.3294j)
(-0.04021-0.04021j)
